In [1]:
using Pkg 
Pkg.instantiate()
using qAlgebra

Precompiling project...
    972.8 ms  ✓ qAlgebra
  1 dependency successfully precompiled in 1 seconds. 7 already precompiled.


In [2]:
qspace = StateSpace("alpha", "beta(t)", "gamma_i", "delta_i", operators=["A(!i)", "B(U,H,i)"], h=QubitPM(), i=(3, QubitPauli()), b=Ladder())

StateSpace: [β(t), γᵢ, γⱼ, γₖ, δᵢ, δⱼ, δₖ, α]
   - SubSpace ["h"]: PM Qubit (Fermionic):  pₚ, mₚ, zₚ, Iₚ (identity)
   - SubSpace ["i", "j", "k"]: Pauli Qubit (Fermionic):  xₚ, yₚ, zₚ, Iₚ (identity)
   - SubSpace ["b"]: Ladder (Bosonic):  p†, p
   - Op: A
   - Op: B(H,U)


In [3]:
var_dict, op_dict, abstract_dict = base_operators(qspace)
alpha = base_operators(qspace, "alpha")
ph, mh, zh = base_operators(qspace, "h", do_dict=false)
xi,yi,zi, pi, mi = base_operators(qspace, "i", do_dict=false)
xj, yj,zj, pj, mj = base_operators(qspace, "j")
I = base_operators(qspace, "I")
A = base_operators(qspace, "A")

A

In [8]:
Sum("i", alpha*ph*xi*yi * A * xi + xi*yi)

izᵢ+iαpₕzᵢAxᵢ

In [4]:
alpha*ph*xi*yi*A

iαpₕzᵢA

In [5]:
a = -alpha*ph*xi*yi * A  + xi*yi

izᵢ-iαpₕzᵢA

In [6]:
a = -alpha*ph*xi*yi * A  + alpha*xi*yi

-iα(pₕzᵢA-zᵢ)

In [4]:
xi*yi

izᵢ

In [5]:
alpha*ph*xi*yi 

iαpₕzᵢ

In [8]:
var_dict, op_dict, abstract_dict = base_operators(qspace)
op_dict

Dict{String, qExpr} with 23 entries:
  "x_i" => xᵢ
  "y_h" => -i(pₕ-mₕ)
  "m_k" => (xₖ+iyₖ)
  "b"   => b
  "x_j" => xⱼ
  "y_k" => yₖ
  "p_h" => pₕ
  "z_i" => zᵢ
  "y_i" => yᵢ
  "z_h" => zₕ
  "x_h" => (pₕ+mₕ)
  "m_h" => mₕ
  "m_j" => (xⱼ+iyⱼ)
  "z_j" => zⱼ
  "x_k" => xₖ
  "m_i" => (xᵢ+iyᵢ)
  "z_k" => zₖ
  "p_k" => (xₖ-iyₖ)
  "I"   => 1
  ⋮     => ⋮

In [ ]:
var_dict

In [3]:
A = base_operators(qspace, "A_2")
alpha = base_operators(qspace, "alpha")
A*alpha*A

α*A₂²

In [4]:
alpha^2*A^2*alpha*A

α³*A₂³